In [5]:
# CÉLULA 1: Extração de Dados (Extract)

import pandas as pd
import os
import sys

# --- VERIFICAÇÃO E INSTALAÇÃO AUTOMÁTICA DE DEPENDÊNCIAS ---
try:
    import gdown
except ImportError:
    print("⚠️ Biblioteca 'gdown' não encontrada. Instalando agora...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])
    import gdown
    print("✅ Biblioteca 'gdown' instalada com sucesso!")

# --- CONFIGURAÇÃO ---
caminho_base = r'C:\Users\Celso\Downloads'
arquivos = {
    2021: 'atendimentoturismo2021.csv',
    2022: 'atendimentoturismo2022.csv',
    2023: 'atendimentoturismo2023.csv'
}
drive_ids = {
    2021: '1sOiEizQLfYH_R2yfuL_6vwm-B4C_rItE',
    2022: '1-0Yeze_VG4dVAgrFluK-a-3kEvryHi40',
    2023: '1EABOwDWcQWnBeD8EfbrBqMQKHxJq6DOb'
}

dfs = []

# Garante diretório
if not os.path.exists(caminho_base):
    try:
        os.makedirs(caminho_base)
        print(f"📁 Pasta criada: {caminho_base}")
    except:
        caminho_base = '.'

print("🚀 Iniciando Extração...")

for ano, nome_arquivo in arquivos.items():
    caminho_completo = os.path.join(caminho_base, nome_arquivo)
    
    # 1. DOWNLOAD (Se não existir localmente)
    if not os.path.exists(caminho_completo):
        file_id = drive_ids.get(ano)
        if file_id:
            print(f"   📥 Baixando arquivo de {ano}...")
            try:
                gdown.download(f'https://drive.google.com/uc?id={file_id}', caminho_completo, quiet=False)
            except Exception as e:
                print(f"   ❌ Falha no download de {ano}: {e}")

    # 2. LEITURA (Se o arquivo existir)
    if os.path.exists(caminho_completo):
        try:
            # Tenta UTF-8
            df_temp = pd.read_csv(caminho_completo, sep=';', encoding='utf-8', on_bad_lines='skip')
            dfs.append(df_temp)
            print(f"   ✅ {ano}: Lido com sucesso ({len(df_temp)} registros)")
        except:
            try:
                # Tenta Latin-1
                df_temp = pd.read_csv(caminho_completo, sep=';', encoding='latin1', on_bad_lines='skip')
                dfs.append(df_temp)
                print(f"   ✅ {ano} (via latin1): Lido com sucesso ({len(df_temp)} registros)")
            except Exception as e:
                print(f"   ❌ Erro crítico ao ler {nome_arquivo}: {e}")
    else:
        print(f"   ⚠️ Arquivo de {ano} indisponível.")

# Consolidação
if dfs:
    df_raw = pd.concat(dfs, ignore_index=True)
    
    # Padronização APENAS do cabeçalho (técnico, para não quebrar o banco)
    df_raw.columns = (df_raw.columns
                      .str.strip().str.lower()
                      .str.replace(' ', '_').str.replace('ç', 'c')
                      .str.replace('ã', 'a').str.replace('ó', 'o')
                      .str.replace('í', 'i').str.replace('ú', 'u'))
    
    print(f"\n📊 Sucesso! {len(df_raw)} linhas prontas na memória ('df_raw').")
else:
    print("\n❌ Falha: Nenhum dado foi extraído.")

🚀 Iniciando Extração...
   ✅ 2021: Lido com sucesso (15374 registros)
   ✅ 2022: Lido com sucesso (37510 registros)
   ✅ 2023: Lido com sucesso (22239 registros)

📊 Sucesso! 75123 linhas prontas na memória ('df_raw').


In [6]:
# CÉLULA 2: Carga na Staging Area (Load)

from sqlalchemy import create_engine
from urllib.parse import quote_plus

if 'df_raw' in locals() and not df_raw.empty:
    print("🐘 Iniciando Carga no PostgreSQL...")
    
    # --- CONFIGURAÇÃO DO BANCO ---
    DB_USER = 'postgres'
    DB_PASS = 'P@celso4364' # Sua senha com @ tratado
    DB_HOST = 'localhost'
    DB_PORT = '5432'
    DB_NAME = 'turismo_recife'
    
    # Tratamento de caractere especial na senha
    encoded_pass = quote_plus(DB_PASS)
    string_conexao = f"postgresql://{DB_USER}:{encoded_pass}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    
    # Nome da tabela temporária (Staging)
    tabela_staging = 'staging_atendimentos_raw'
    
    try:
        engine = create_engine(string_conexao)
        
        print(f"   📤 Enviando {len(df_raw)} registros para '{tabela_staging}'...")
        
        # Carga em lotes (Chunksize) para performance
        df_raw.to_sql(tabela_staging, engine, if_exists='replace', index=False, chunksize=10000)
        
        print("🎉 SUCESSO! Dados brutos carregados no banco.")
        print("   -> Próximo passo: Rodar transformações via SQL.")
        
    except Exception as e:
        print(f"\n❌ Erro na conexão ou carga: {e}")
        print("   Dica: Verifique se o banco 'turismo_recife' existe.")
        
else:
    print("❌ Erro: O DataFrame 'df_raw' não existe. Rode o Bloco 1 primeiro.")

🐘 Iniciando Carga no PostgreSQL...
   📤 Enviando 75123 registros para 'staging_atendimentos_raw'...
🎉 SUCESSO! Dados brutos carregados no banco.
   -> Próximo passo: Rodar transformações via SQL.


In [7]:
# CÉLULA 3: Transformação, Limpeza e Modelagem (Script SQL Completo)

from sqlalchemy import text

if 'engine' in locals():
    print("🏗️ Executando Script SQL de Transformação e Modelagem...")
    print("   Isso pode levar alguns segundos, pois o banco está processando tudo...")
    
    # Script SQL Único e Robusto
    script_sql_elt = """
    -- ============================================================================ 
-- ETL COMPLETO (SCRIPT ATUALIZADO) - COPIAR E COLAR INTEIRO
-- Versão: Passo 5 otimizado (sem ROW_NUMBER ORDER BY pesado; chaves sintéticas)
-- ============================================================================

BEGIN;

-- BLOCO 1: LIMPEZA ESTRUTURAL (REMOVER LIXO)
DELETE FROM staging_atendimentos_raw 
WHERE (ano IS NULL OR ano = 0) OR (mes IS NULL OR mes = 0);

ALTER TABLE staging_atendimentos_raw 
DROP COLUMN IF EXISTS qtdadoleslentes,
DROP COLUMN IF EXISTS qtdadultos,
DROP COLUMN IF EXISTS qtdcriancas,
DROP COLUMN IF EXISTS qtdidosos,
DROP COLUMN IF EXISTS cidade,
DROP COLUMN IF EXISTS deslocamento,
DROP COLUMN IF EXISTS informacao,
DROP COLUMN IF EXISTS motivoviagem,
DROP COLUMN IF EXISTS observacao,
DROP COLUMN IF EXISTS ehacompanhante,
DROP COLUMN IF EXISTS faixaetaria,
DROP COLUMN IF EXISTS qtdacompanhantes,
DROP COLUMN IF EXISTS qtdturistas;

-- BLOCO 2: PADRONIZAÇÃO GLOBAL (UPPER + TRIM)
UPDATE staging_atendimentos_raw
SET 
    paisorigem        = UPPER(TRIM(paisorigem)),
    estadoorigem      = UPPER(TRIM(estadoorigem)),
    nacionalidade     = UPPER(TRIM(nacionalidade)),
    tipohospedagem    = UPPER(TRIM(tipohospedagem)),
    tipotransporte    = UPPER(TRIM(tipotransporte)),
    municipointeresse = UPPER(TRIM(municipointeresse)),
    localatendimento  = UPPER(TRIM(localatendimento)),
    tipoatendimento   = UPPER(TRIM(tipoatendimento)),
    sexo              = UPPER(TRIM(sexo)),
    tempoestadia      = UPPER(TRIM(tempoestadia));

-- BLOCO 3: SUBSTITUIÇÃO GLOBAL POR 'DESCONHECIDO'
UPDATE staging_atendimentos_raw
SET 
    paisorigem = CASE WHEN paisorigem IN ('NAO INFORMADO', 'NAO PREENCHEU', 'NAO-INFORMOU', 'NAO_INFORMADO', 'SEM INFORMAÇÃO', 'SEM INFORMACAO', 'NAN', 'NULL', '') OR paisorigem IS NULL THEN 'DESCONHECIDO' ELSE paisorigem END,
    estadoorigem = CASE WHEN estadoorigem IN ('NAO INFORMADO', 'NAO PREENCHEU', 'NAO-INFORMOU', 'NAO_INFORMADO', 'SEM INFORMAÇÃO', 'SEM INFORMACAO', 'NAN', 'NULL', '') OR estadoorigem IS NULL THEN 'DESCONHECIDO' ELSE estadoorigem END,
    nacionalidade = CASE WHEN nacionalidade IN ('NAO INFORMADO', 'NAO PREENCHEU', 'NAO-INFORMOU', 'NAO_INFORMADO', 'SEM INFORMAÇÃO', 'SEM INFORMACAO', 'NAN', 'NULL', '') OR nacionalidade IS NULL THEN 'DESCONHECIDO' ELSE nacionalidade END,
    tipohospedagem = CASE WHEN tipohospedagem IN ('NAO INFORMADO', 'NAO PREENCHEU', 'NAO-INFORMOU', 'NAO_INFORMADO', 'SEM INFORMAÇÃO', 'SEM INFORMACAO', 'NAN', 'NULL', '') OR tipohospedagem IS NULL THEN 'DESCONHECIDO' ELSE tipohospedagem END,
    tipotransporte = CASE WHEN tipotransporte IN ('NAO INFORMADO', 'NAO PREENCHEU', 'NAO-INFORMOU', 'NAO_INFORMADO', 'SEM INFORMAÇÃO', 'SEM INFORMACAO', 'NAN', 'NULL', '') OR tipotransporte IS NULL THEN 'DESCONHECIDO' ELSE tipotransporte END,
    municipointeresse = CASE WHEN municipointeresse IN ('NAO INFORMADO', 'NAO PREENCHEU', 'NAO-INFORMOU', 'NAO_INFORMADO', 'SEM INFORMAÇÃO', 'SEM INFORMACAO', 'NAN', 'NULL', '') OR municipointeresse IS NULL THEN 'DESCONHECIDO' ELSE municipointeresse END,
    localatendimento = CASE WHEN localatendimento IN ('NAO INFORMADO', 'NAO PREENCHEU', 'NAO-INFORMOU', 'NAO_INFORMADO', 'SEM INFORMAÇÃO', 'SEM INFORMACAO', 'NAN', 'NULL', '') OR localatendimento IS NULL THEN 'DESCONHECIDO' ELSE localatendimento END,
    tipoatendimento = CASE WHEN tipoatendimento IN ('NAO INFORMADO', 'NAO PREENCHEU', 'NAO-INFORMOU', 'NAO_INFORMADO', 'SEM INFORMAÇÃO', 'SEM INFORMACAO', 'NAN', 'NULL', '') OR tipoatendimento IS NULL THEN 'DESCONHECIDO' ELSE tipoatendimento END,
    sexo = CASE WHEN sexo IN ('NAO INFORMADO', 'NAO PREENCHEU', 'NAO-INFORMOU', 'NAO_INFORMADO', 'SEM INFORMAÇÃO', 'SEM INFORMACAO', 'NAN', 'NULL', '') OR sexo IS NULL THEN 'DESCONHECIDO' ELSE sexo END,
    tempoestadia = CASE WHEN tempoestadia IN ('NAO INFORMADO', 'NAO PREENCHEU', 'NAO-INFORMOU', 'NAO_INFORMADO', 'SEM INFORMAÇÃO', 'SEM INFORMACAO', 'NAN', 'NULL', '') OR tempoestadia IS NULL THEN 'DESCONHECIDO' ELSE tempoestadia END;

-- BLOCO 4: NORMALIZAÇÃO DO TEMPO (CRIAÇÃO E CÁLCULO)
ALTER TABLE staging_atendimentos_raw ADD COLUMN IF NOT EXISTS tempo_maximo_estadia INT;

UPDATE staging_atendimentos_raw
SET tempo_maximo_estadia = (
    CASE 
        WHEN tempoestadia IN ('DESCONHECIDO', 'OUTROS') THEN 0
        WHEN tempoestadia LIKE '%SEMANA%' THEN 
            CAST(SUBSTRING(tempoestadia FROM '(\d+)(?!.*\d)') AS INT) * 7
        WHEN tempoestadia LIKE '%MES%' OR tempoestadia LIKE '%MÊS%' THEN 
            CAST(SUBSTRING(tempoestadia FROM '(\d+)(?!.*\d)') AS INT) * 30
        WHEN tempoestadia LIKE '%ANO%' THEN 
            CAST(SUBSTRING(tempoestadia FROM '(\d+)(?!.*\d)') AS INT) * 365
        WHEN tempoestadia ~ '[0-9]' THEN 
            CAST(SUBSTRING(tempoestadia FROM '(\d+)(?!.*\d)') AS INT)
        ELSE 0 
    END
);

-- Ajuste de memória para sessão
SET work_mem = '512MB';

-- Índices na origem (ajuda no planejamento dos joins)
CREATE INDEX IF NOT EXISTS idx_stg_tempo ON staging_atendimentos_raw(ano, mes);
CREATE INDEX IF NOT EXISTS idx_stg_local ON staging_atendimentos_raw(paisorigem, estadoorigem, nacionalidade);
CREATE INDEX IF NOT EXISTS idx_stg_viagem ON staging_atendimentos_raw(tipohospedagem, tipotransporte, municipointeresse, tempoestadia);
CREATE INDEX IF NOT EXISTS idx_stg_atend ON staging_atendimentos_raw(localatendimento, tipoatendimento);

ANALYZE staging_atendimentos_raw;

-- ============================
-- PASSO 5 OTIMIZADO (SIMPLIFICADO) - CHAVES SINTÉTICAS E DIMENSÕES
-- ============================

-- 5.0: criar chaves sintéticas na staging para joins rápidos
ALTER TABLE staging_atendimentos_raw
    ADD COLUMN IF NOT EXISTS key_localizacao TEXT,
    ADD COLUMN IF NOT EXISTS key_viagem TEXT,
    ADD COLUMN IF NOT EXISTS key_atendimento TEXT;

UPDATE staging_atendimentos_raw
SET
    key_localizacao = COALESCE(TRIM(paisorigem),'<>') || '|' || COALESCE(TRIM(estadoorigem),'<>') || '|' || COALESCE(TRIM(nacionalidade),'<>'),
    key_viagem      = COALESCE(TRIM(tipohospedagem),'<>') || '|' || COALESCE(TRIM(tipotransporte),'<>') || '|' || COALESCE(TRIM(municipointeresse),'<>') || '|' || COALESCE(TRIM(tempoestadia),'<>'),
    key_atendimento = COALESCE(TRIM(localatendimento),'<>') || '|' || COALESCE(TRIM(tipoatendimento),'<>');

CREATE INDEX IF NOT EXISTS idx_stg_key_loc   ON staging_atendimentos_raw(key_localizacao);
CREATE INDEX IF NOT EXISTS idx_stg_key_via   ON staging_atendimentos_raw(key_viagem);
CREATE INDEX IF NOT EXISTS idx_stg_key_atend ON staging_atendimentos_raw(key_atendimento);
CREATE INDEX IF NOT EXISTS idx_stg_ano_mes   ON staging_atendimentos_raw(ano, mes);

ANALYZE staging_atendimentos_raw;

-- 5.1 Dimensão Calendário (sem ORDER BY pesado)
DROP TABLE IF EXISTS dim_calendario CASCADE;
CREATE UNLOGGED TABLE dim_calendario (
    id_tempo SERIAL PRIMARY KEY,
    ano INT,
    mes INT,
    key_tempo TEXT
);
INSERT INTO dim_calendario (ano, mes, key_tempo)
SELECT DISTINCT ano, mes, COALESCE(ano::text,'0') || '|' || COALESCE(mes::text,'0')
FROM staging_atendimentos_raw
WHERE ano IS NOT NULL;
CREATE INDEX IF NOT EXISTS idx_dim_cal_join ON dim_calendario(ano, mes);
ANALYZE dim_calendario;

-- 5.2 Dimensão Localização (sem ORDER BY pesado)
DROP TABLE IF EXISTS dim_localizacao CASCADE;
CREATE UNLOGGED TABLE dim_localizacao (
    id_localizacao SERIAL PRIMARY KEY,
    paisorigem TEXT,
    estadoorigem TEXT,
    nacionalidade TEXT,
    key_localizacao TEXT
);
INSERT INTO dim_localizacao (paisorigem, estadoorigem, nacionalidade, key_localizacao)
SELECT DISTINCT paisorigem, estadoorigem, nacionalidade, key_localizacao
FROM staging_atendimentos_raw;
CREATE INDEX IF NOT EXISTS idx_dim_loc_key ON dim_localizacao(key_localizacao);
ANALYZE dim_localizacao;

-- 5.3 Dimensão Viagem (sem ORDER BY pesado)
DROP TABLE IF EXISTS dim_viagem CASCADE;
CREATE UNLOGGED TABLE dim_viagem (
    id_viagem SERIAL PRIMARY KEY,
    tipohospedagem TEXT,
    tipotransporte TEXT,
    municipointeresse TEXT,
    tempoestadia TEXT,
    tempo_maximo_estadia INT,
    key_viagem TEXT
);
INSERT INTO dim_viagem (tipohospedagem, tipotransporte, municipointeresse, tempoestadia, tempo_maximo_estadia, key_viagem)
SELECT DISTINCT tipohospedagem, tipotransporte, municipointeresse, tempoestadia, tempo_maximo_estadia, key_viagem
FROM staging_atendimentos_raw;
CREATE INDEX IF NOT EXISTS idx_dim_via_key ON dim_viagem(key_viagem);
ANALYZE dim_viagem;

-- 5.4 Dimensão Atendimento (sem ORDER BY pesado)
DROP TABLE IF EXISTS dim_atendimento CASCADE;
CREATE UNLOGGED TABLE dim_atendimento (
    id_atendimento_info SERIAL PRIMARY KEY,
    localatendimento TEXT,
    tipoatendimento TEXT,
    key_atendimento TEXT
);
INSERT INTO dim_atendimento (localatendimento, tipoatendimento, key_atendimento)
SELECT DISTINCT localatendimento, tipoatendimento, key_atendimento
FROM staging_atendimentos_raw;
CREATE INDEX IF NOT EXISTS idx_dim_atend_key ON dim_atendimento(key_atendimento);
ANALYZE dim_atendimento;

-- 5.5 FATO (criado em um único passo, usando chaves indexadas)
DROP TABLE IF EXISTS fato_atendimentos CASCADE;

CREATE UNLOGGED TABLE fato_atendimentos AS
SELECT
    s.idatendimento,
    c.id_tempo,
    l.id_localizacao,
    v.id_viagem,
    a.id_atendimento_info,
    s.sexo
FROM staging_atendimentos_raw s
JOIN dim_calendario c    ON s.ano = c.ano AND s.mes = c.mes
JOIN dim_localizacao l   ON s.key_localizacao = l.key_localizacao
JOIN dim_viagem v        ON s.key_viagem = v.key_viagem
JOIN dim_atendimento a   ON s.key_atendimento = a.key_atendimento;

ALTER TABLE fato_atendimentos ADD PRIMARY KEY (idatendimento);
ANALYZE fato_atendimentos;

-- LIMPEZA FINAL (opcional: remover colunas auxiliares da staging)
-- ALTER TABLE staging_atendimentos_raw DROP COLUMN IF EXISTS key_localizacao, DROP COLUMN IF EXISTS key_viagem, DROP COLUMN IF EXISTS key_atendimento;

COMMIT;
    
    """
    
    # Executa o script no banco
    with engine.connect() as con:
        con.execute(text(script_sql_elt))
        con.commit()
        
    print("\n✅ SQL Executado com Sucesso!")
    print("   -> Limpeza realizada na Staging.")
    print("   -> Dimensões criadas: dim_calendario, dim_localizacao, dim_viagem, dim_atendimento.")
    print("   -> Fato criada: fato_atendimentos.")
    
else:
    print("❌ Conexão com o banco não encontrada. Rode o Bloco 2 primeiro.")

<>:10: SyntaxWarning: invalid escape sequence '\d'
<>:10: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Celso\AppData\Local\Temp\ipykernel_5088\1731329122.py:10: SyntaxWarning: invalid escape sequence '\d'
  script_sql_elt = """


🏗️ Executando Script SQL de Transformação e Modelagem...
   Isso pode levar alguns segundos, pois o banco está processando tudo...

✅ SQL Executado com Sucesso!
   -> Limpeza realizada na Staging.
   -> Dimensões criadas: dim_calendario, dim_localizacao, dim_viagem, dim_atendimento.
   -> Fato criada: fato_atendimentos.
